Customer segementation plays an important role to identify which group a company should focus on in retention by promotion, which group is a main source of company's profit. There are several techniques including K Means Clustering, EM (Expectation Maximization). While K Means are commonly, it suffers a drawback. The algorithm works well when a boundary between clusters is clearly determined. **In other word**, one data point should belongs to one cluster or not straddle in seperated line. K-Means uses Euclidean metrics that illustrates well a sphere boundary with a centered centroid

Real world data may hardly be perfect this way. EM considers the data point belongs to one cluster if the probability of its memebership is maximum. This means EM computes the probability of membership for any data point belongs to each of clusters. In this notebook, we explorehow to use EM approach to segment music (Jazz) into categories. We adapt from [source]("https://www.google.com/search?ei=kVoLWoWvH8LqjwPUjL6QBA&q=thoughtful+machine+learning+with+python&oq=thoughtful+machine+learning+with+python&gs_l=psy-ab.3..35i39k1j0i67k1l2j0l6.1126.2246.0.2668.8.8.0.0.0.0.117.809.4j4.8.0....0...1.1.64.psy-ab..0.8.805....0.CnGd_QoiG0A")


In [1]:
from collections import namedtuple
import random
import logging
import math

import numpy as np
from numpy.linalg import LinAlgError


In [3]:
from collections import namedtuple
import random
import logging
import math

import numpy as np
from numpy.linalg import LinAlgError


def dvmnorm(x, mean, covariance, log=False):
    """density function for the multivariate normal distribution
    based on sources of R library 'mvtnorm'
    :rtype : np.array
    :param x: vector or matrix of quantiles. If x is a matrix, each row is taken to be a quantile
    :param mean: mean vector, np.array
    :param covariance: covariance matrix, np.array
    :param log: if True, densities d are given as log(d), default is False
    """
  # TODO: add another methods from mvtnorm (calculate matrix square root using eigenvalues or SVD
  # TODO: add check for matching of input matrix dimensions
    n = covariance.shape[0]
    try:
        dec = np.linalg.cholesky(covariance)
    except LinAlgError:
        dec = np.linalg.cholesky(covariance + np.eye(covariance.shape[0]) * 0.0001)
    tmp = np.linalg.solve(dec, np.transpose(x - mean))
    rss = np.sum(tmp * tmp, axis=0)
    logretval = - np.sum(np.log(np.diag(dec))) - 0.5 * n * np.log(2 * math.pi) - 0.5 * rss
    if log:
        return logretval
    else:
        return np.exp(logretval)


class EMClustering(object):
    logger = logging.getLogger(__name__)
    ch = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    ch.setFormatter(formatter)
    logger.addHandler(ch)
    logger.setLevel(logging.DEBUG)

    cluster = namedtuple('cluster', 'weight, mean, covariance')

    def __init__(self, n_clusters):
        self._data = None
        self._clusters = None
        self._membership_weights = None
        self._partitions = None
        self._n_clusters = n_clusters

    @property
    def partitions(self):
        return self._partitions

    @property
    def data(self):
        return self._data

    @property
    def labels(self):
        return self._membership_weights

    @property
    def clusters(self):
        return self._clusters

    def fit_predict(self, data, iteration=5):
        self.setup(data)
        for i in range(iteration):
            self.logger.debug('Iteration %d', i)
            self.expect()
            self.maximize()
        return self

    def setup(self, data):
        self._n_samples, self._n_features = data.shape
        self._data = data
        self._membership_weights = np.ones((self._n_samples, self._n_clusters)) / self._n_clusters
        self._s = 0.2

        indices = list(range(data.shape[0]))
        random.shuffle(indices)
        pick_k_random_indices = random.sample(indices, self._n_clusters)

        self._clusters = []
        for cluster_num in range(self._n_clusters):
            mean = data[pick_k_random_indices[cluster_num], :]
            covariance = self._s * np.eye(self._n_features)
            self._clusters.append(self.cluster(1.0 / self._n_clusters, mean, covariance))

        self._partitions = np.empty(self._n_samples, dtype=np.int32)

    def expect(self):
        log_likelyhood = 0
        for cluster_num, cluster in enumerate(self._clusters):
            log_density = dvmnorm(self._data, cluster.mean, cluster.covariance, log=True)
            membership_weights = cluster.weight * np.exp(log_density)
            log_likelyhood += sum(log_density * self._membership_weights[:, cluster_num])

            self._membership_weights[:, cluster_num] = membership_weights

        for sample_num, probabilities in enumerate(self._membership_weights):
            prob_sum = sum(probabilities)

            self._partitions[sample_num] = np.argmax(probabilities)

            if prob_sum == 0:
                self._membership_weights[sample_num, :] = np.ones_like(probabilities) / self._n_clusters
            else:
                self._membership_weights[sample_num, :] = probabilities / prob_sum

        self.logger.debug('log likelyhood %f', log_likelyhood)

    def maximize(self):
        for cluster_num, cluster in enumerate(self._clusters):
            weights = self._membership_weights[:, cluster_num]

            weight = np.average(weights)
            mean = np.average(self._data, axis=0, weights=weights)
            covariance = np.cov(self._data, rowvar=False, ddof=0, aweights=weights)

            self._clusters[cluster_num] = self.cluster(weight, mean, covariance)


In [9]:
import os
import csv
import numpy as np
# In case the above code stored in em_clustering.py, uncomment the following
#from em_clustering import EMClustering

In [ ]:
data = []
artists = []
years = []

In [ ]:
## Read data from csv with open, Do Not use
with open('C:/dataset/annotated_jazz_albums.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    headers = reader.fieldnames[3:]
    for row in reader:
        artists.append(row['artist_album'])
        years.append(row['year'])
        data.append([int(row[key]) for key in headers])

In [14]:
#Alternative, we can do 
# assert data= df.iloc[:,3:].values
data= df.iloc[:,3:].values

1378

In [16]:
# Here we use pandas 
import pandas as pd
df = pd.read_csv("data/annotated_jazz_albums.csv")
df.head()

,artist_album,key_index,year,Aboriginal,Abstract,Acid Jazz,Acoustic,African,Afro-Cuban,Afro-Cuban Jazz,...,Swing,Synth-pop,Tech House,Techno,Theme,Therapy,Thrash,Trance,Trip Hop,Vocal
0,Duke Ellington In A Mellotone,0,1940,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,Duke Ellington Sophisticated Lady,1,1940,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,Duke Ellington Black Brown and Beige,2,1943,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Coleman Hawkins Rainbow Mist,3,1944,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,Mary Lou Williams Zodiac Suite,4,1945,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#Alternative, we can do 
# assert data= df.iloc[:,3:].values
data= df.iloc[:,3:].values

(1378, 130)

In [18]:
np.array(data)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [10]:
clusterer = EMClustering(n_clusters=25)
clusters = clusterer.fit_predict(np.array(data))

print(clusters.partitions)

2017-11-14 12:48:20,375 - __main__ - DEBUG - Iteration 0
2017-11-14 12:48:22,965 - __main__ - DEBUG - log likelyhood -30145.611290
2017-11-14 12:48:23,245 - __main__ - DEBUG - Iteration 1
2017-11-14 12:48:23,465 - __main__ - DEBUG - log likelyhood 415701.263098
2017-11-14 12:48:23,838 - __main__ - DEBUG - Iteration 2
2017-11-14 12:48:24,134 - __main__ - DEBUG - log likelyhood 592838.564443
2017-11-14 12:48:24,307 - __main__ - DEBUG - Iteration 3
2017-11-14 12:48:24,526 - __main__ - DEBUG - log likelyhood 600303.866821
2017-11-14 12:48:24,697 - __main__ - DEBUG - Iteration 4
2017-11-14 12:48:24,915 - __main__ - DEBUG - log likelyhood 602965.110886


[ 2  2 13  2 13  4  2  4 13  4 13 13  4  4 23  4  4  4  2 13  4  2 13 13  4
  4 13  4  2  4  4 13  8 13  2 13  4  4  4 13 13 13  4 13 13 13 13  4  4 13
  2 21 13  4  4  4  4  4  4  4  3  4 13  4 13  4  4 13  4  4 13 23  4 13  4
 13 23  4 13  4  4 13  4  3 13 13  4 13 13 14  3  4  4 13  4 13  4  4  4 13
 13  4  4  4 21 13 13 21 13 13 23  4 13  4 13  4 13 13 13  4  4  4 23 10 13
  4  4 13  6  4 21  4 13  4 16 13 13 13 13  4 13  4  4 13  4  4 23  4  4 23
  2  4 13 14  1 13 14 13 13 13  4  4  4 13 13 13 13 13  2 13  3  3 13 13  4
 23 13  4  4 13  4  4 23  4  4 13 23 13 23 13  8  4 13 13 13 13 13 13  4  4
  4 13 15  2  9 13  4  4  4 13  3 13 13  4 13 13 13  4  3 13  4 13 13 13 13
 13  4 13 13 13 16 13 13  8  8  4 23 13  4 13 13  4 13  4 23 13  3  3 13 13
 13 13 13 13 13 13  2  4 23 16 13 13 13 13 13 13 13  3  4 13 23 13  4 23  4
 13 13  8 23 13 14 11 13 13  4 13 13 13 13 13 13  4 13 13 13  4 13 13  8 13
 13 13 13  8 13 13 13 13  4 13 23 13 21  8 13 13  4 13  4  4 13 13 13  0 23
 11  8  8 13

Note that there is a drawback, each run we may get different clusters

In [42]:
from sklearn.cluster import KMeans
clusters = KMeans(n_clusters=25).fit_predict(data)


with open('clustered_kmeans.csv', 'w') as csvfile:
    fieldnames = ['artist_album', 'year', 'cluster']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for i, cluster in enumerate(clusters):
        writer.writerow({'artist_album': artists[i], 'year': years[i],'cluster': cluster})